# LH5, DataGroup, and DSP Optimization Tutorial

Clint Wiseman, UW.  Presented at [LEGEND Software Tutorial, Nov. 2021](https://indico.legend-exp.org/event/561/)

With Ge and SiPM detectors, we need to be able to **look at waveforms,and quickly find optimal DSP parameters to get the best energy resolution and pulse shape discrimination**.  

Users should be able to run this notebook to manually search for "best guess" parameters. 
You can select a group of files to analyze using the `fileDB.h5` and a pandas query for a particular set of files.  This notebook only needs the **raw LH5 files,** and requires the user just manually select the **1460 and 2615** keV peaks, using the onboard energy parameter from the Struck card. 

Here are a few things we want to do in this notebook:

- Show an example of `DataGroup`, which is frequently used for detector test stand setups (CAGE, HADES, SURF, etc.)
- Give examples of **creating and reading LH5 files**, which are required by pygama's `ProcessingChain` DSP class.
- **Optimize pole-zero corrections** to flatten the waveforms in `[4250:5500], [4250:8000]`
- **Optimize energy trapezoid** integration and flat top times (including asymmetric trap) to get best energy resolution
- **Optimize DCR parameter** -- vary the windows used to maximize alpha/gamma separation
- TODO: Optimize energy with charge trapping correction (see notes at the end)

In [ ]:
import os, sys, h5py, json, time
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

# use this to get interactive plots at NERSC.  
# requires ipympl & jupyter-matplotlib extension for jupyterlab
# user may need to $pip install ipympl --update
%matplotlib widget

import matplotlib.pyplot as plt

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    from tqdm import tqdm
    tqdm.pandas() # suppress annoying FutureWarning

import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
from pygama import DataGroup, lh5
from pygama.dsp.dsp_optimize import *
from pygama.dsp.WaveformBrowser import WaveformBrowser as wfb

In [ ]:
# << -- CONFIG -- >> # 

# set query
# query = "run == 280" # "2185-2212", "alp", "elog 364. overnight alpha run"
query = 'cycle > 2185 and cycle < 2188'

# set raw energy estimator
etype = 'energy'
tb_in = 'ORSIS3302DecoderForEnergy/raw'
wf_in = 'ORSIS3302DecoderForEnergy/raw/waveform'
xlo, xhi, xpb = 0, 4e6, 10000

# set rough calibration: user fills this in from the first plot, below
peaks = {
    '40K':  [1460.8, 1.71e6],
    '208TL':[2614.5, 3.06e6]
}

# set up a WaveformBrowser for just one of the peaks (user-selected)
pk_select = '208TL'
# pk_select = '40K'

In [ ]:
# load data from the raw files.
# we need cycle number and packet index information to retreive waveforms, so make a dataframe

dg = DataGroup('./metadata/cage.json', load=True)
dg.fileDB.query(query, inplace=True)

if len(dg.fileDB)==0:
    print('Error, no files found.  Check your query, and fileDB.h5.')
    dg.fileDB[['runtype', 'run', 'cycle']]

def load_file(row):
    data_cols = [etype, 'ievt']
    raw_file = dg.lh5_dir + row['raw_path'] + '/' + row['raw_file']
    raw_cycle = row.cycle
    raw_data = pd.DataFrame(lh5.load_nda(raw_file, data_cols, tb_in, verbose=False))
    raw_data['cycle'] = row['cycle']
    return raw_data
        
result = dg.fileDB.progress_apply(load_file, axis=1)
df_data = pd.concat([r for r in result])
df_mem = round(sys.getsizeof(df_data) / 1024 / 1024, 2)

print('Found unique cycles:', df_data['cycle'].unique())
print(f"Entries found: {df_data.shape}")
print('In-memory size:', df_mem, "MB")

raw_files = dg.lh5_dir + dg.fileDB['raw_path'] + '/' + dg.fileDB['raw_file']

with h5py.File(raw_files.iloc[0], 'r') as hf:
    print('LH5 columns found :', list(hf[f'{tb_in}'].keys()))
    
df_data

In [ ]:
# create raw energy histogram
%matplotlib widget
hist, bins, var = pgh.get_hist(df_data['energy'], range=(xlo, xhi), dx=xpb)
bins = bins[1:] # trim zero bin, not needed with ds='steps'
plt.semilogy(bins, hist, ds='steps', c='b', lw=1, label=etype)
plt.xlabel(etype)
plt.ylabel(f'cts, {xpb}/bin')
plt.show()
print('Hey, you need to zoom in and pick out the locations of the 1460 and 2615 peaks and write them into the config at the top!')

In [ ]:
%matplotlib widget

# select events in peaks using a rough FWHM, optimized for onboard energy.
# this should work automatically if 'peaks' is correct.
# a lot of this is reused from CAGE energy_cal.py :: fit_peaks

evts_pass = {}

# adjustable parameters for auto-binning
win_multip = 30
nbin_gain = 4
nevt_max = 5000 # num. waveforms to retrieve from each peak
nsig_save = 4 # num. sig to save.  may need to be able to re-fit the whole peak

fig, axs = plt.subplots(1, len(peaks), figsize=(len(peaks)*5, 4))

for ipk, (pk, (cal_e, raw_e)) in enumerate(peaks.items()):
    
    # set the window.  assume resolution goes as roughly sqrt(energy).
    window = np.sqrt(raw_e) * win_multip
    plo, phi = raw_e - window / 2, raw_e + window / 2
    nbin_scale = nbin_gain * np.sqrt(raw_e) / raw_e
    nbins = int(window) * nbin_scale
    ppb = (phi - plo) / nbins
    
    h, b, var = pgh.get_hist(df_data[etype], range=(plo, phi), dx=ppb)
    b = b[1:]
    
    # get fwhm and select events in the window (ix_evts).  don't need to fit here.
    imax = np.argmax(h)
    ix_upr = np.where((b > b[imax]) & (h <= np.amax(h)/2))
    ix_bot = np.where((b < b[imax]) & (h <= np.amax(h)/2))
    
    upr_half = b[ix_upr][0]
    bot_half = b[ix_bot][-1]
    fwhm0 = upr_half - bot_half
    sig0 = fwhm0 / 2.355

    raw_ctr = b[imax]
    raw_lo = raw_ctr - sig0 * nsig_save
    raw_hi = raw_ctr + sig0 * nsig_save

    # select 'nevt_max' events to save waveforms, but sample evenly throughout the dataset
    df_pass = df_data.loc[(df_data[etype] > raw_lo) & (df_data[etype] < raw_hi)]
    n_evts = len(df_pass)
    n_select = nevt_max if n_evts > nevt_max else n_evts
    evts_pass[pk] = df_pass.sample(n_select).sort_values(['cycle','ievt'])
    # print(evts_pass[pk])
    
    axs[ipk].plot(b, h, ds='steps', c='b', lw=1, label=pk + ', ' + etype)
    axs[ipk].axvline(raw_lo, c='r', alpha=0.3, label=f'{nsig_save}-sig window\n{n_evts} cts')
    axs[ipk].axvline(raw_hi, c='r', alpha=0.3)
    axs[ipk].legend(fontsize=8, loc=2)
    axs[ipk].set_xlabel(etype)

In [ ]:
# save waveforms for events in peaks into memory as an LH5 table (and to disk).
# this can take a long time, which is why it's nice to only run once.

sto = lh5.Store()

tb_wfs = {} # LH5 tables for each peak

# write the data to a temporary LH5 file usable by WaveformBrowser
f_wfs = './metadata/wfs_optimizer.lh5' 
if os.path.exists(f_wfs):
    os.remove(f_wfs) # write_object :: append=False can't create a new file

for pk, df_evts in evts_pass.items():
    print(f'Loading {pk} peak data ...')
    
    raw_files = dg.lh5_dir + dg.fileDB['raw_path'] + '/' + dg.fileDB['raw_file']
    raw_files = list(raw_files)
    
    df_idxs = df_evts.groupby('cycle').agg({'ievt' : lambda x: list(x)})
    raw_idxs = df_idxs['ievt'].values
    raw_idxs = list(raw_idxs)
    
    # this step can take a while, so time it
    t_start = time.time()
    tb_wfs[pk], n_wfs = sto.read_object(tb_in, raw_files, idx=raw_idxs)
    t_elap = (time.time() - t_start) / 60
    print(tb_wfs[pk]['waveform']['values'].nda.shape)
    print(f'    Waveforms loaded.  Elapsed time: {t_elap:.2f} min.')
    
    # some handy debug statements looking at h5py dset attrs, etc
    # tb_wfs[pk].attrs['datatype'] = 'table{t0, dt, values}' # required by WaveformBrowser
    # print(dir(tb_wfs[pk]))
    # print(tb_wfs.keys())
    # tb_raw = lh5.Table(col_dict={'waveform':tb_wfs[pk]})
    # tb_raw.attrs['datatype'] = 'table{waveform}'
    
    sto.write_object(tb_wfs[pk], f'{tb_in}/{pk}', f_wfs, append=True)

In [ ]:
# For our output file to work with WaveformBrowser, the LH5 Tables, and their
# HDF5 attrs etc, all need to be the same.  To make sure this is the case,
# compare against the first raw file in our list, which we know is readable
# by WaveformBrowser.

with h5py.File(f_wfs, 'r') as hf:
    print('LH5 columns found :', list(hf[f'{tb_in}/{pk_select}'].keys()))
    
def print_attrs(name, obj):
    print(name)
    for key, val in obj.attrs.items():
        print("    %s: %s" % (key, val))
        
# debug -- this is useful for looking at LH5 dataset & attributes
        
# f_input = raw_files[0]
# with h5py.File(f_input, 'r') as hf:
#     print('LH5 columns found :', list(hf[f'{tb_in}'].keys()))
#     hf.visititems(print_attrs)

print('\nnew wfs file:')
f_input = f_wfs
with h5py.File(f_input, 'r') as hf:
    print('LH5 columns found :', list(hf[f'{tb_in}/{pk_select}'].keys()))
    hf.visititems(print_attrs)

In [ ]:
%matplotlib widget

# show some example waveforms from all peaks before running WaveformBrowser and ParGrid.
# limit to the rising edge because it's more interesting.

n_max = 100

fig, axs = plt.subplots(1, len(peaks), figsize=(len(peaks)*5, 4))

for ipk, (pk, (cal_e, raw_e)) in enumerate(peaks.items()):
    
    n_wfs, nsamp = tb_wfs[pk]['waveform']['values'].nda.shape
    n_lim = n_max if n_wfs > n_max else n_wfs
    wfs = tb_wfs[pk]['waveform']['values'].nda[:n_lim,:]
    
    wf_ctr = nsamp / 2
    win_lo, win_hi = int(wf_ctr - 200), int(wf_ctr + 200)
    
    ts = np.arange(0, len(wfs[0, win_lo:win_hi]))
    for iwf in range(wfs.shape[0]):
        axs[ipk].plot(ts, wfs[iwf,win_lo:win_hi], lw=2, alpha=0.5)
    axs[ipk].plot(np.nan, np.nan, label = f'{pk}, {n_lim} wfs')
    axs[ipk].legend(loc=4, fontsize=10)

plt.show()

### 1. Optimize Pole-Zero Correction ('whole-tail')

Here, I want to select values for the pole-zero correction (using the `double_pole_zero` calculator) that give the whole tail after the rising edge as flat of a slope as possible.  The tasks here are:

- Set up a minimal DSP config file 
- Declare a `WaveformBrowser` example to show the pole-zero corrected waveform, so that users can draw various wfs
- Set up a `ParGrid` class and call `run_one_dsp` for ONE set of parameters, to look at how the **figure of merit** works.
- Then in `optimizer_v6_grid.py`, we call the same FOM and run on a larger grid search.

**NOTE:** We really don't need very many waveforms to get the pole-zero correction correct (almost all the tails have the same decay constant).  So we can limit this to like 10-100 waveforms and the `ParGrid` search will run much faster.

In [ ]:
# declare the DSP processor list.  later cells will augment these config parameter dict's.

dsp_config = {
    "outputs" : ["bl", "bl_sig", "pztail_mean", "pztail_sig"],
    "processors" : {
         "bl , bl_sig, slope, intercept":{
            "function": "linear_slope_fit",
            "module": "pygama.dsp.processors",
            "args" : ["waveform[:1650]", "bl","bl_sig", "slope","intercept"],
            "unit": ["ADC","ADC","ADC","ADC"]
        },
        "wf_blsub":{
            "function": "subtract",
            "module": "numpy",
            "args": ["waveform", "bl", "wf_blsub"],
            "prereqs": ["waveform", "bl"],
            "unit": "ADC",
        },
        "wf_pz": {
            "function": "double_pole_zero",
            "module": "pygama.dsp.processors",
            "prereqs": ["wf_blsub"],
            "args": ["wf_blsub", "db.pz2.tau1", "db.pz2.tau2",  "db.pz2.frac", "wf_pz"],
            "defaults": {"db.pz2.tau1":"187.5*us", "db.pz2.tau2":"3.17*us", "db.pz2.frac":"0.035" },
            "unit" : "ADC"
        },
         "pztail_mean , pztail_sig, pz_slope, pz_intercept":{
            "function": "linear_slope_fit",
            "module": "pygama.dsp.processors",
            "args" : ["wf_pz[4250:8000]", "pztail_mean","pztail_sig", "pz_slope","pz_intercept"],
            "unit": ["ADC","ADC","ADC","ADC"]
        },
    }
}

# try writing to a temporary file
# f_dsp = './dsp_tmp.json'
# with open(f_dsp, 'w') as f:
#     json.dump(dsp_config, f)

# user should MANUALLY edit the parameters of interest to get an idea
# of which range we should pick to optimize with ParGrid.
# this DB will be updated with the 'optimized' parameter choices
# (consider this as an output of this notebook).
# note: description of the formatting is in pygama.dsp.build_processing_chain
dsp_db = {
    "pz2" : {
        "tau1" : "51*us",
        "tau2" : "2*us",
        "frac" : 0.04
    }
}

In [ ]:
## -- waveform browser step -- 

# load our skim waveforms file
f_input = f_wfs 
tb_input = f'{tb_in}/{pk_select}'

# debug - load wf browser with raw file 0 & preexisting config
# dsp_config = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_06.json')
# f_input = raw_files[0] # testing, has correct datatypes
# tb_input = f'{tb_in}'

pprint(dsp_db)

b = wfb(f_input, tb_input, dsp_config,
        waveforms=['wf_blsub', 'wf_pz'],
        database=dsp_db,
        # selection = cut,
        wf_styles=[{'linestyle':['-']}, {'linestyle':[':']}],
        #legend=['wf_blsub {bl:.2f}', 'pz-corrected'], # displaying the bl value is tricky
        legend_opts={'loc':"lower right"},
        #lines=['bl'],
        x_lim=(38000, 80000)
       )

In [ ]:
%matplotlib widget
b.draw_next()

In [ ]:
%matplotlib widget

# -- EXAMPLE -- run_one_dsp and evaluate a figure of merit.
# This is automated by the ParGrid class, but it's good to be able to visualize & make plots of what's happening.

col_name = 'pztail_sig'

def mean_val(tb, verbosity):
    return np.average(tb[col_name].nda)

# limit the number of wfs, we only need ~10--50 to get the pz correction right.  
# it's easier to reload from the file than to try and slice the in-memory `tb_wfs[pk_select]`.
nwfs_lim = 10
sto = lh5.Store()
tb_wfs_slim, n_wfs = sto.read_object(f'{tb_in}/{pk_select}', f_wfs, n_rows=nwfs_lim)

tb_out = run_one_dsp(tb_wfs_slim, dsp_config, db_dict=dsp_db, verbosity=1, )

mean = mean_val(tb_out, 0)
xlo, xhi, xpb = mean - mean/2, mean + mean/2, 0.5
h, b, var = pgh.get_hist(tb_out[col_name].nda, range=(xlo, xhi), dx=xpb)
plt.plot(b[1:], h, ds='steps', c='b')
plt.xlabel(col_name)
plt.show()

In [ ]:
# ParGrid setup
# internally this calls run_one_dsp with our figure of merit function.
# In the notebook we don't want to run on too many points,
# We can run a really FINE grid search in `optimizer_v6_grid.py` on the batch system.

pg = ParGrid()

# vary tau1, tau2, and frac
tau1_arr = np.linspace(51, 52, 6)
tau2_arr = np.linspace(5.5, 7, 6)
frac_arr = np.linspace(0.03, 0.05, 4)

pg.add_dimension('wf_pz', 1, [f"{t:.2f}*us" for t in tau1_arr])
pg.add_dimension('wf_pz', 2, [f"{t:.2f}*us" for t in tau2_arr])
pg.add_dimension('wf_pz', 3, [f"{t:.3f}" for t in frac_arr])

print('tau1:', tau1_arr)
print('tau2:', tau2_arr)
print('frac:', frac_arr)
ngrid = pg.get_n_grid_points()
print('grid points to search:', ngrid)

# the more waveforms we have, the longer it will take to run one grid point
nwfs = tb_wfs_slim['waveform']['values'].nda.shape[0]
print('wfs to reprocess:', nwfs * ngrid)

In [ ]:
# run the grid search.

# NOTE: the fom_function does NOT support additional function arguments.
fom_vals = run_grid(tb_wfs_slim, dsp_config, pg, mean_val, verbosity=0)

# unpack the results into a DataFrame.  
# have to iterate thru the n-dimensional grid
grid_nd = []
ix = pg.get_zero_indices()
while True:
    row = []
    for i_dim, i_par in enumerate(ix):
        name, i_arg, value_str, _ = pg.get_data(i_dim, i_par)
        #if '*' in value_str:
        #    val = float(value_str.split('*')[0])
        #else:
        #    val = float(value_str)
        row.append(value_str)
    grid_nd.append(row)
    if not pg.iterate_indices(ix): break

df_grid = pd.DataFrame(grid_nd, columns=['tau1','tau2','frac'])

results_1d = fom_vals.reshape(-1, pg.get_n_grid_points())
df_grid['fom'] = results_1d[0]

print("NOTE: if one of the best settings is at the upper/lower limit of your parameter grid,",
      "\nyou probably need to adjust the grid to find the true min.")

# df_grid # show full df
df_best = df_grid.sort_values('fom')[:5] # show 5 best settings
df_best

In [ ]:
# finally, update the dsp_db with the best result

dbest = df_best.iloc[0].to_dict()

for par, val in dbest.items():
    if par == 'fom': continue
    dsp_db['pz2'][par] = val

print("NOTE: you can go back and re-run the WaveformBrowser step now,",
     "\nto see the effect of the updated PZ values.")
dsp_db

## 2. Energy Trapezoid Optimization

Our main energy estimator is `trapEftp`, which uses a combination of different trapezoid settings.  The most important ones to get right are in the  "energy trapezoid" `wf_trap` -- the trap filter with a long integration time.  This is used for `trapEmax`.  The fixed-time-pickoff calculation also uses an asymmetric trapezoid to find `t0`, and then we set a "pickoff time".  Jason and Ian have said that as long as the pickoff time selects a point on the flat top of the energy trapezoid, its value is rather arbitrary.  Similarly, the `t0` calculator is fairly robust too, and these parameters shouldn't need to be varied as much as the energy trap parameters.

So here, we're going to optimize `db.etrap.rise`, `db.etrap.flat`, and for fun we can also optimize `db.pz2.tau1` to see if we get a different value than what we got above.  I bet we will ...

In [ ]:
# re-declare dsp_config to contain energy trapezoid calculators.
# this is visually simpler than trying to augment the existing dictionary.
# however, we should update the dsp_db dict, since it's the "output" of this notebook.

dsp_config = {
    "outputs" : ["tp_0", "trapEmax", "atrap_max", "trapEftp", "tp_ftp"],
    "processors" : {
         "bl , bl_sig, slope, intercept":{
            "function": "linear_slope_fit",
            "module": "pygama.dsp.processors",
            "args" : ["waveform[:1650]", "bl","bl_sig", "slope","intercept"],
            "unit": ["ADC","ADC","ADC","ADC"]
        },
        "wf_blsub":{
            "function": "subtract",
            "module": "numpy",
            "args": ["waveform", "bl", "wf_blsub"],
            "prereqs": ["waveform", "bl"],
            "unit": "ADC",
        },
        "wf_pz": {
            "function": "double_pole_zero",
            "module": "pygama.dsp.processors",
            "prereqs": ["wf_blsub"],
            "args": ["wf_blsub", "db.pz2.tau1", "db.pz2.tau2",  "db.pz2.frac", "wf_pz"],
            "defaults": {"db.pz2.tau1":"187.5*us", "db.pz2.tau2":"3.17*us", "db.pz2.frac":"0.035" },
            "unit" : "ADC"
        },
        "wf_etrap": {
            "function": "trap_norm",
            "module": "pygama.dsp.processors",
            "prereqs": ["wf_pz"],
            "args": ["wf_pz", "db.etrap.rise", "db.etrap.flat", "wf_etrap"],
            "defaults" : {"db.etrap.rise":"4*us", "db.etrap.flat":"1*us"},
            "unit": "ADC"
        },
        "wf_atrap": {
            "function": "asym_trap_filter",
            "module": "pygama.dsp.processors",
            "prereqs": ["wf_pz"],
            # "args": ["wf_pz", "round(0.1*us)", "round(1*us)", "round(4*us)", "wf_atrap"], # ian's
            "args": ["wf_pz", "db.atrap.rise", "db.atrap.flat", "db.atrap.fall", "wf_atrap"], # clint's
            "defaults" : {"db.atrap.rise":"20*ns", "db.atrap.flat":"1*us","db.atrap.fall":"4*us"},
            "unit": "ADC"
        },
        "trapEmax": {
            "function": "amax",
            "module": "numpy",
            "args": ["wf_etrap", 1, "trapEmax"],
            "kwargs": {"signature":"(n),()->()", "types":["fi->f"]},
            "unit": "ADC",
            "prereqs": ["wf_etrap"]
        },
         "atrap_max": {
             "function": "argmax",
              "module": "numpy",
              "args": ["wf_atrap", 1, "atrap_max"],
              "kwargs": {"signature":"(n),()->()", "types":["fi->i"]},
              "unit": "ADC",
              "prereqs": ["wf_atrap"]
        },
        "tmax": {
            "function": "argmax",
            "module": "numpy",
            "args": ["wf_atrap", 1, "tmax"],
            "kwargs": {"signature":"(n),()->()", "types":["fi->i"]},
            "unit": "ns"
        },
        "tp_0": {
            "function": "time_point_thresh",
            "module": "pygama.dsp.processors",
            "args": ["wf_atrap", 0, "tmax", 0, "tp_0"],
            "unit": "ns",
        },
         "trapEftp": {
            "function": "fixed_time_pickoff",
            "module": "pygama.dsp.processors",
            "args": ["wf_etrap", "db.tp_ftp.ftp", "trapEftp"],
            "defaults" : {"db.tp_ftp.ftp":"tp_0 + 5.5*us"},
            "unit": "ADC",
            "prereqs": ["wf_etrap", "tp_0"]
        },
        "tp_ftp" : {
            "function":"add",
            "module":"numpy",
            "args":["tp_0", "db.tp_ftp.ftp", "tp_ftp"],
            "defaults" : {"db.tp_ftp.ftp":"tp_0 + 5.5*us"},
            "prereqs":["tp_0"],
            "unit":"ns"
        }
    }
}

# add parameters to dsp_db
dsp_db['etrap'] = {"rise":"4*us", "flat":"2*us"}
dsp_db['atrap'] = {"rise":"20*ns", "flat":"1*us", "fall":"4*us"}
dsp_db['tp_ftp'] = {"ftp":"tp_0 + 5.5*us"} # << this gets trapEftp correct!!
# dsp_db['tp_ftp'] = {"ftp":"5.5*us"} # << this one draws correctly on the plot !! weird.

# pprint(dsp_db)

In [ ]:
## -- waveform browser step -- 
# user should MANUALLY edit the parameters of interest to get an idea
# of which ranges & parameters we should pick to optimize with ParGrid

# load our skim waveforms file
f_input = f_wfs 
tb_input = f'{tb_in}/{pk_select}'

print('DB input parameters:')
pprint(dsp_db)

b = wfb(f_input, tb_input, dsp_config,
        waveforms=['wf_blsub', 'wf_pz', 'wf_etrap', 'wf_atrap'],
        database=dsp_db,
        legend=['wf_blsub', 'wf_pz', 'wf_etrap', 'wf_atrap'],
        # lines=['trapEftp', 'tp_ftp', 'tp_0'], # hmm, I can't get any of these to work, need to ask ian
        x_lim=(38000, 55000)
       )

In [ ]:
%matplotlib widget
b.draw_next()

In [ ]:
%matplotlib widget

# -- EXAMPLE: try a couple different figures of merit to optimize the
# height/width of our test peak.

# ene_type = 'trapEftp' # this one was giving me issues
ene_type = 'trapEmax' # <-- need to use this one

win_multip = 0.5
nbin_gain = 100

def peak_height(tb, verbosity):
    raw_e = np.mean(tb[ene_type].nda)
    window = np.sqrt(raw_e) * win_multip
    plo, phi = raw_e - window / 2, raw_e + window / 2
    nbin_scale = nbin_gain * np.sqrt(raw_e) / raw_e
    nbins = int(window) * nbin_scale
    ppb = (phi - plo) / nbins
    h, b, var = pgh.get_hist(tb[ene_type].nda, range=(plo, phi), dx=ppb)
    # plt.plot(b[1:], h, ds='steps')
    # plt.show()
    return np.max(h)


def peak_width(tb, verbosity, make_plot=False):
    
    # histogram the data
    raw_e = np.mean(tb[ene_type].nda)
    window = np.sqrt(raw_e) * win_multip
    plo, phi = raw_e - window / 2, raw_e + window / 2
    nbin_scale = nbin_gain * np.sqrt(raw_e) / raw_e
    nbins = int(window) * nbin_scale
    ppb = (phi - plo) / nbins
    h, bins, var = pgh.get_hist(tb[ene_type].nda, range=(plo, phi), dx=ppb)
    b = bins[1:]

    # get initial guesses for simple Gauss fit
    imax = np.argmax(h)
    ix_upr = np.where((b > b[imax]) & (h <= np.amax(h)/2))
    ix_bot = np.where((b < b[imax]) & (h <= np.amax(h)/2))
    upr_half = b[ix_upr][0]
    bot_half = b[ix_bot][-1]
    fwhm0 = upr_half - bot_half
    sig0 = fwhm0 / 2.355
    amp0 = np.amax(h) * fwhm0
    p_init = [b[imax], sig0, amp0]

    # run curve_fit through pygama's wrapper function
    fit_func = pgf.gauss
    p_fit, p_cov = pgf.fit_hist(fit_func, h, bins,
                                var=var, guess=p_init)
    p_err = np.sqrt(np.diag(p_cov))

    if make_plot:
        plt.plot(b, h, ds='steps', c='b', lw=1, label=pk_select + ', ' + etype)
        xfit = np.arange(plo, phi, ppb * 0.1)
        plt.plot(xfit, fit_func(xfit, *p_init), '-', c='orange', label='init')
        plt.plot(xfit, fit_func(xfit, *p_fit), '-', c='red', label='fit')
        plt.legend(fontsize=8, loc=2)
        plt.xlabel(etype)
        plt.show()

    return p_fit[1] * 2.355 # fwhm

print('DB parameters:')
pprint(dsp_db)

# run dsp -- for some reason it's not printing the db.etrap lookup, but seems to use it ...
tb_out = run_one_dsp(tb_wfs[pk_select], dsp_config, db_dict=dsp_db, verbosity=1)

# this is how i examined the problem with tp_ftp and trapEftp 
print(tb_out.keys())
df = tb_out.get_dataframe()

# check figure of merit
height = peak_height(tb_out, 0)
print("counts in max bin:", height)

fwhm = peak_width(tb_out, 0, True)
print("fwhm:", fwhm)

df

In [ ]:
# ParGrid setup
# internally this calls run_one_dsp with our figure of merit function.
# In the notebook we don't want to run on too many points,
# We can run a really FINE grid search in `optimizer_v6_grid.py` on the batch system.

pg = ParGrid()

# vary rise, flat, and pz
rise_arr = np.linspace(8, 11, 9)
flat_arr = np.linspace(2, 4, 3)
# pz_arr = np.linspace(51, 51, 1)

pg.add_dimension('wf_etrap', 1, [f"{t:.2f}*us" for t in rise_arr])
pg.add_dimension('wf_etrap', 2, [f"{t:.2f}*us" for t in flat_arr])
# pg.add_dimension('wf_pz', 3, [f"{t:.2f}*us" for t in pz_arr])

print('rise:', rise_arr)
print('flat:', flat_arr)
# print('pz:', pz_arr)
ngrid = pg.get_n_grid_points()
print('grid points to search:', ngrid)

# the more waveforms we have, the longer it will take to run one grid point
nwfs = tb_wfs[pk_select]['waveform']['values'].nda.shape[0]
print('wfs to reprocess:', nwfs * ngrid)

In [ ]:
# run the grid search.

# NOTE: the fom_function does NOT support additional function arguments.
fom_vals = run_grid(tb_wfs[pk_select], dsp_config, pg, peak_width, db_dict=dsp_db, verbosity=0)

# unpack the results into a DataFrame.  
# have to iterate thru the n-dimensional grid
grid_nd = []
ix = pg.get_zero_indices()
while True:
    row = []
    for i_dim, i_par in enumerate(ix):
        name, i_arg, value_str, _ = pg.get_data(i_dim, i_par)
        #if '*' in value_str:
        #    val = float(value_str.split('*')[0])
        #else:
        #    val = float(value_str)
        row.append(value_str)
    grid_nd.append(row)
    if not pg.iterate_indices(ix): break

df_grid = pd.DataFrame(grid_nd, columns=['rise','flat'])

results_1d = fom_vals.reshape(-1, pg.get_n_grid_points())
df_grid['fom'] = results_1d[0]

print("NOTE: if one of the best settings is at the upper/lower limit of your parameter grid,",
      "\nyou probably need to adjust the grid to find the true min.")

# df_grid # show full df
df_best = df_grid.sort_values('fom')
df_best[:5]

In [ ]:
# update the dsp_db with the best result.
print(df_best.iloc[0])
dbest = df_grid.sort_values('fom').iloc[0].to_dict()

for par, val in dbest.items():
    if par == 'fom': continue
    dsp_db['etrap'][par] = val
    
# tp_ftp should be updated to tp_0 + rise + flat/2 automatically, or you're gonna forget ...
rt = float(dsp_db['etrap']['rise'].split('*')[0])
ft = float(dsp_db['etrap']['flat'].split('*')[0])
dsp_db['tp_ftp']['ftp'] = f'tp_0 + {rt+ft/2}*us'

print("NOTE 1: you can go back and re-run the WaveformBrowser step now,",
     "\nto see the effect of the updated values.")

dsp_db



In [ ]:
%matplotlib widget

# run dsp with the best result and check resolution

print('DB parameters:')
pprint(dsp_db)
# pprint(dsp_config)

# run dsp
tb_out = run_one_dsp(tb_wfs[pk_select], dsp_config, db_dict=dsp_db, verbosity=1)

# this is how i examined the problem with tp_ftp and trapEftp 
print(tb_out.keys())
df = tb_out.get_dataframe()


# check figure of merit
height = peak_height(tb_out, 0)
print("counts in max bin:", height)

fwhm = peak_width(tb_out, 0, True)
print("fwhm:", fwhm)

df

In [ ]:
# load our skim waveforms file
f_input = f_wfs 
tb_input = f'{tb_in}/{pk_select}'

print('DB input parameters:')
pprint(dsp_db)

b = wfb(f_input, tb_input, dsp_config,
        waveforms=['wf_blsub', 'wf_pz', 'wf_etrap', 'wf_atrap'],
        database=dsp_db,
        legend=['wf_blsub', 'wf_pz', 'wf_etrap', 'wf_atrap'],
        lines=['tp_ftp', 'tp_0', 'trapEftp', 'tp_ftp'],
        x_lim=(38000, 80000)
       )

In [ ]:
%matplotlib widget
b.draw_next()

In [ ]:
pprint(dsp_db)

## 3. DCR Parameter Optimization

DCR is basically measuring "the slope of the tail" of each waveform.  We know that alpha events tend to have a higher slope, meaning that some excess charge is "trickling in" to the detector much more slowly than the bulk of the charge collection, which creates the rising edge of a signal.  To calculate it, instead of fitting the tail (which is slow), we just take the average in two ~1 us windows on the tail, and subtract them: `DCR = win2 - win1 / len(win)`

In pygama this is equivalent to calculating a trapezoid filter with `rise = win1, win2` and `flat = (spacing between windows)` (typically 10--20 usec), and then evaluating the trapezoid at its first point (a fixed-time pickoff).

In previous CAGE analyses, we've found that varying the window where we optimize the pole-zero correction (so, making another choice than `[4200:8000]` for the tail) can lead to better alpha/gamma event separation.  To optimize DCR, there are sort of two steps:
1. Take a small population of gamma events (so, our 40K or 208TL sample waveforms) and tune `dcr` such that it's as close to zero as possible.  Note there may be an overall trend with energy that you can't really get rid of.
2. Check the alpha-gamma separation in a 2D plot.  Access the first few files in the `raw_files` array to get enough statistics.  This can also take a few minutes to complete `run_one_dsp` so keeping the array sizes as small as possible is useful.

In [ ]:
dsp_config = {
    "outputs" : ["tp_0", "trapEmax", "atrap_max", "trapEftp", "tp_ftp", "dcr"],
    "processors" : {
         "bl , bl_sig, slope, intercept":{
            "function": "linear_slope_fit",
            "module": "pygama.dsp.processors",
            "args" : ["waveform[:3500]", "bl","bl_sig", "slope","intercept"],
            "unit": ["ADC","ADC","ADC","ADC"]
        },
        "wf_blsub":{
            "function": "subtract",
            "module": "numpy",
            "args": ["waveform", "bl", "wf_blsub"],
            "prereqs": ["waveform", "bl"],
            "unit": "ADC",
        },
        "wf_pz": {
            "function": "double_pole_zero",
            "module": "pygama.dsp.processors",
            "prereqs": ["wf_blsub"],
            "args": ["wf_blsub", "db.pz2.tau1", "db.pz2.tau2",  "db.pz2.frac", "wf_pz"],
            "defaults": {"db.pz2.tau1":"187.5*us", "db.pz2.tau2":"3.17*us", "db.pz2.frac":"0.035" },
            "unit" : "ADC"
        },
        "wf_etrap": {
            "function": "trap_norm",
            "module": "pygama.dsp.processors",
            "prereqs": ["wf_pz"],
            "args": ["wf_pz", "db.etrap.rise", "db.etrap.flat", "wf_etrap"],
            "defaults" : {"db.etrap.rise":"4*us", "db.etrap.flat":"1*us"},
            "unit": "ADC"
        },
"wf_atrap": {
            "function": "asym_trap_filter",
            "module": "pygama.dsp.processors",
            "prereqs": ["wf_pz"],
            # "args": ["wf_pz", "round(0.1*us)", "round(1*us)", "round(4*us)", "wf_atrap"], # ian's
            "args": ["wf_pz", "db.atrap.rise", "db.atrap.flat", "db.atrap.fall", "wf_atrap"], # clint's
            "defaults" : {"db.atrap.rise":"20*ns", "db.atrap.flat":"1*us","db.atrap.fall":"4*us"},
            "unit": "ADC"
        },
        "trapEmax": {
            "function": "amax",
            "module": "numpy",
            "args": ["wf_etrap", 1, "trapEmax"],
            "kwargs": {"signature":"(n),()->()", "types":["fi->f"]},
            "unit": "ADC",
            "prereqs": ["wf_etrap"]
        },
         "atrap_max": {
             "function": "argmax",
              "module": "numpy",
              "args": ["wf_atrap", 1, "atrap_max"],
              "kwargs": {"signature":"(n),()->()", "types":["fi->i"]},
              "unit": "ADC",
              "prereqs": ["wf_atrap"]
        },
        "tmax": {
            "function": "argmax",
            "module": "numpy",
            "args": ["wf_atrap", 1, "tmax"],
            "kwargs": {"signature":"(n),()->()", "types":["fi->i"]},
            "unit": "ns"
        },
        "tp_0": {
            "function": "time_point_thresh",
            "module": "pygama.dsp.processors",
            "args": ["wf_atrap", 0, "tmax", 0, "tp_0"],
            "unit": "ns",
        },
         "trapEftp": {
            "function": "fixed_time_pickoff",
            "module": "pygama.dsp.processors",
            "args": ["wf_etrap", "db.tp_ftp.ftp", "trapEftp"],
            "defaults" : {"db.tp_ftp.ftp":"tp_0 + 5.5*us"},
            "unit": "ADC",
            "prereqs": ["wf_etrap", "tp_0"]
        },
        "tp_ftp" : {
            "function":"add",
            "module":"numpy",
            "args":["tp_0", "db.tp_ftp.ftp", "tp_ftp"],
            "defaults" : {"db.tp_ftp.ftp":"tp_0 + 5.5*us"},
            "prereqs":["tp_0"],
            "unit":"ns"
        },
        "wf_pz_dcr": {
            "function": "double_pole_zero",
            "module": "pygama.dsp.processors",
            "prereqs": ["wf_blsub"],
            "args": ["wf_blsub", "db.dcr_pz.tau1", "db.dcr_pz.tau2",  "db.dcr_pz.frac", "wf_pz_dcr"],
            "defaults": {"db.dcr_pz.tau1":"50*us", "db.dcr_pz.tau2":"6*us", "db.dcr_pz.frac":"0.04" },
            "unit" : "ADC"
        },
        "wf_dcr_trap": {
            "function": "trap_norm",
            "module": "pygama.dsp.processors",
            "args": ["wf_pz_dcr", "db.dcr_trap.rise", "db.dcr_trap.flat", "wf_dcr_trap"],
            "defaults" : {"db.dcr_trap.rise":"7.5*us", "db.dcr_trap.flat":"22.5*us"},
            "unit": "ADC",
            "prereqs": ["wf_pz_dcr"]
        },
        "dcr": {
            "function": "fixed_time_pickoff",
            "module": "pygama.dsp.processors",
            "args": ["wf_dcr_trap", "db.dcr.ftp", "dcr"],
            "defaults" : {"db.dcr.ftp" : "79*us"},
            "unit": "ADC",
            "prereqs": ["wf_dcr_trap"]
        }
    }
}

# add parameters to dsp_db
# dsp_db['dcr_pz'] = {"tau1":"52*us", "tau2":"6.*us", "frac":"0.04"}
dsp_db['dcr_pz'] = dsp_db['pz2'] # set equal to best result from above
# dsp_db['dcr_trap'] = {"rise":"7.5*us", "flat":"22.5*us"}
dsp_db['dcr_trap'] = {'flat': '20.33*us', 'rise': '7.00*us'}
dsp_db['dcr'] = {"ftp":"80*us"}

pprint(dsp_db)

In [ ]:
# browse waveforms

f_input = f_wfs 
tb_input = f'{tb_in}/{pk_select}'

print('DB input parameters:')
pprint(dsp_db)

b = wfb(f_input, tb_input, dsp_config,
        waveforms=['wf_blsub', 'wf_pz_dcr', 'wf_dcr_trap'],
        database=dsp_db,
        # legend=['wf_blsub', 'wf_pz_dcr', 'wf_dcr_trap'],
        lines=['dcr'],
        x_lim=(38000, 80000)
       )

In [ ]:
%matplotlib widget
b.draw_next()

In [ ]:
%matplotlib widget

# EXAMPLE -- run_one_dsp with this config file and check the output dataframe.

def mean_val(tb, verbosity, make_plot=False):
    
    if make_plot:
        ps = pd.Series(tb['dcr'].nda)
        ps.hist()
    
    return np.average(tb["dcr"].nda)

print('DB parameters:')
pprint(dsp_db)

# run dsp -- for some reason it's not printing the db.etrap lookup, but seems to use it ...
tb_out = run_one_dsp(tb_wfs[pk_select], dsp_config, db_dict=dsp_db, verbosity=1)

print(tb_out.keys())
df = tb_out.get_dataframe()

# check figure of merit
mean_dcr = mean_val(tb_out, 0, True)
print("mean:", mean_dcr)

df

In [ ]:
# ParGrid setup


pg = ParGrid()

# we can vary: dcr_pz.tau1, dcr_pz.tau2, dcr_trap.rise, dcr_trap.flat,
# but the most sensitive parameters should be tau1 and tau2.
# NOTE: we also may want to vary the waveform window, which can only be set in dsp_config

# optimizing tau didn't really do that well, it introduced a lot of curvature.
# seems like it's better to use the best-fit pz corrected waveform found above,
# and then vary rise/flat to try to pin the dcr values of 2615 to 0 ...
# tau1_arr = np.linspace(50, 53, 10)
# tau2_arr = np.linspace(2, 4, 3)
# pg.add_dimension('wf_pz_dcr', 1, [f"{t:.2f}*us" for t in tau1_arr])
# pg.add_dimension('wf_pz_dcr', 2, [f"{t:.2f}*us" for t in tau2_arr])
# print('tau1:', tau1_arr)
# print('tau2:', tau2_arr)

rise_arr = np.linspace(6, 10, 5)
flat_arr = np.linspace(15, 21, 10)
print(rise_arr)
print(flat_arr)

pg.add_dimension('wf_dcr_trap', 1, [f"{t:.2f}*us" for t in rise_arr])
pg.add_dimension('wf_dcr_trap', 2, [f"{t:.2f}*us" for t in flat_arr])

ngrid = pg.get_n_grid_points()
print('grid points to search:', ngrid)

# the more waveforms we have, the longer it will take to run one grid point
nwfs = tb_wfs[pk_select]['waveform']['values'].nda.shape[0]
print('wfs to reprocess:', nwfs * ngrid)

In [ ]:
# run the grid search.

# NOTE: the fom_function does NOT support additional function arguments.
fom_vals = run_grid(tb_wfs[pk_select], dsp_config, pg, mean_val, db_dict=dsp_db, verbosity=0)

# unpack the results into a DataFrame.  
# have to iterate thru the n-dimensional grid
grid_nd = []
ix = pg.get_zero_indices()
while True:
    row = []
    for i_dim, i_par in enumerate(ix):
        name, i_arg, value_str, _ = pg.get_data(i_dim, i_par)
        row.append(value_str)
    grid_nd.append(row)
    if not pg.iterate_indices(ix): break

df_grid = pd.DataFrame(grid_nd, columns=['rise','flat'])

results_1d = fom_vals.reshape(-1, pg.get_n_grid_points())
df_grid['fom'] = results_1d[0]

print("NOTE: if one of the best settings is at the upper/lower limit of your parameter grid,",
      "\nyou probably need to adjust the grid to find the true min.")

# df_grid # show full df
df_best = df_grid.sort_values('fom', key=abs)
df_best[:5]

In [ ]:
# update the dsp_db with the best result.
print(df_best.iloc[0])
dbest = df_grid.sort_values('fom', key=abs).iloc[0].to_dict()

dbest

for par, val in dbest.items():
    if par == 'fom': continue
    dsp_db['dcr_trap'][par] = val
    
print("NOTE: you can go back and re-run the WaveformBrowser step now,",
     "\nto see the effect of the updated values.")

dsp_db



In [ ]:
%matplotlib widget

# run dsp with the best result and check the central value of DCR

print('DB parameters:')
pprint(dsp_db)
# pprint(dsp_config)

# run dsp
tb_out = run_one_dsp(tb_wfs[pk_select], dsp_config, db_dict=dsp_db, verbosity=1)

# print(tb_out.keys())
df = tb_out.get_dataframe()
# print(df)

# make a quick histogram of the dcr column.  pandas auto-histogram for a column should be enough
df.hist('dcr', bins=int(len(df) * 0.1))
plt.show()

In [ ]:
# load our skim waveforms file
f_input = f_wfs 
tb_input = f'{tb_in}/{pk_select}'

print('DB input parameters:')
pprint(dsp_db)

b = wfb(f_input, tb_input, dsp_config,
        waveforms=['wf_blsub', 'wf_pz', 'wf_pz_dcr', 'wf_dcr_trap'],
        database=dsp_db,
        # legend=['wf_blsub', 'wf_pz', 'wf_etrap', 'wf_atrap'],
        # lines=['tp_ftp', 'tp_0', 'trapEftp', 'tp_ftp'],
        x_lim=(38000, 80000)
       )

In [ ]:
%matplotlib widget
b.draw_next()

In [ ]:
pprint(dsp_db)

In [ ]:
#NOTE: now you need to make the 2D plot with DCR vs Energy to see alpha/gamma separation.
# it can take 3-4 minutes to process each file.  
# to speed up the time it takes to search the parameter space, we apply a pretty aggressive low-e cut.
# we can come back and process the full file at the end.

%time
nlim = 1
files_in = raw_files[:nlim]

# run dsp on the full file.  can take a while ... 2-3 mins
print('DB parameters:')
pprint(dsp_db)
# pprint(dsp_config)

# apply low-e cut
# raw_data = pd.DataFrame(lh5.load_nda(raw_file, data_cols, tb_in, verbose=False))
raw_e = lh5.load_nda(raw_files[:nlim], ['energy'], tb_in, verbose=False)

print(type(raw_e['energy']))

ix = np.where(raw_e['energy'] > 1e6)[0] # this is somewhere between 583 and 1460. 
print(type(ix))

print(type(raw_files[:nlim]))

# read the waveforms.  this can take quite a while (3--4 minutes)
tb_data, n_wfs = sto.read_object(tb_in, raw_files[:nlim], idx=ix)
print(n_wfs)

In [ ]:
# DEBUG line -- manually vary dsp_db

# dsp_db['dcr_pz']['tau1'] = "60*us" # best: 52*us
# dsp_db['dcr_pz']['tau2'] = "6.4*us" # best: 6.4*us
# dsp_db['dcr_pz']['frac'] = "0.043" # best: 0.043
# dsp_db['dcr_trap']['rise'] = "8*us"
# dsp_db['dcr_trap']['flat']= "23*us"
# dsp_db['dcr']['ftp'] = "80*us"

# dcr_trap rise/flat maxes: [10, 20], [9, 22]

# run the dsp
tb_out = run_one_dsp(tb_data, dsp_config, db_dict=dsp_db, verbosity=0)
print(tb_out.keys())
df_data = tb_out.get_dataframe()
# df_data

In [ ]:
%matplotlib widget
from matplotlib.colors import LogNorm

# 2D plot of DCR vs trapEftp

xlo, xhi, xpb = 0, 15000, 10
ylo, yhi, ypb = -200, 200, 1
# ylo, yhi, ypb = -600, 0, 1

nbx = int((xhi-xlo)/xpb)
nby = int((yhi-ylo)/ypb)

plt.hist2d(df_data.trapEftp, df_data.dcr, range=((xlo,xhi),(ylo,yhi)), bins=(nbx, nby), 
           cmap='jet', norm=LogNorm())

plt.xlabel('trapEftp')
plt.ylabel('dcr')
plt.show()

In [ ]:
# finally, write the DB to an output text file.

pprint(dsp_db)

# try writing to a temporary file
f_dsp = './metadata/optimizer_results.json'
with open(f_dsp, 'w') as f:
    json.dump(dsp_db, f)



## 4. Charge Trapping Correction

Two methods:
* Drift Time
* DCR
    
FIRST, you want to optimize the energy trapezoids & pole zero consts by themselves.  
(That's what all the work up to this point was for!)
THEN, you can improve the resolution further by introducing a corrected parameter.

1. DT method:
Fixed time pickoff should be the same as the ramp time, started at t0.  
Second region must be totally in the flat top, first region starts at t0
The correction should be: `E_new = E + const * qDrift`, but need to check it works for multiple peaks.
It might also be `E_new = E * (1 + const * qDrift)`

2. DCR method: 
DCR-modified E: `E_new = E + const * DCR`
The reason this is as effective as the drift time method is that delayed charge collection from the **bulk** is released w. some time constant on the order of the digitizer window length.  This will make it show up in the tail of the waveforms.  Of course, alpha events have delayed collection from the **surface** as well, which should have a different time constant.

In MJD, charges are significantly trapped but released before next event due to the lower operating temperature, 77 K.
In the STCs and CAGE, the operating temp is closer to 85 or 90 K, and the charge re-release time will be faster. 
So we will have to decide for ourselves if Method 1 (DT) or Method 2 (DCR) is a more effective charge trapping correction for us.

In [ ]:
# lol TBD  :-)